In [1]:
# Hidden TimeStamp
import time, datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
print('Last Run: {}'.format(st))

Last Run: 2016-09-15 19:53:00


# The Package Architecture

LamAna originates from a single legacy script (circa 2014).  It has since grown into a package of modules and been abstracted to address more general problems related to laminate analysis.  

This repository is designed to analyze various geometries given a specified custom model based on Classical Laminate Theory (CLT).  Package architecture is diagramed below:  

![API Diagram](./_images/diagram.png)

As indicated in the diagram key, each diamond shape represents a module.  This diagram also illustrates the traffic of data structures exchanged between modules. Namely, these data structures are `FeatureInput` and `LaminateModels` objects; details for each will be described later.  The user-related areas are highlighted blue.  The most prominent extensible components pertain to these blue areas.  

## Package Module Summary

The following table summarizes the core and feature modules in this package, their intended functions and key resulting objects.  Objects that are  exchanged between modules are italicized.  The Auxillary/Utility modules house supporting code that will not be discussed.

| Module | Classifier | Purpose     | Product |
|:------ |:---------- |:----------- |:-------:|
| `input_` | Backend   | Backend code processing user inputs for all feature modules.  | User *Input object* i.e. `Geometry` |
| `distrubtions` | Feature | Analyze stress distributions for different geometries.  | *FeatureInput object* | 
| `ratios` | Feature | Thickness ratio analyses for optimizing stress-geomtry design. | |
| `predictions` | Feature | Failure predictions using experimental and laminate theory data. | |
| `constructs` | Backend | Build DataFrame representations of laminates. | *Laminate-like objects* |
| `theories` | Backend |  Handle custom model selection and handshaking. |  |
| `<models>` | Extension | Directory of user-defined, custom LT models | *Model object* |
| `output_` | Backend |  Plotting objects and export code. | Output object e.g. plots, .xlsx/.csv, figures |

The components of the lamana project can thus be classified by one of three types:

- **Frontend/Feature**: user-interacted, feature modules of particular interest that rely on models based on laminate theory.
- **Extension**: plugin modules that extend capabilities of the repository, e.g. `models` directory containing user defined laminate theories (`Classical_LT`, `Wilson_LT`).
- **Backend**: remaining Core modules, `input_`, `constructs_`, `theories_`, `output_`; workhorse factories of `LaminateModel` objects.

### Intramodular Products

Intramodular products exchange information between package modules.  These objects are illustrated as circles in the API Diagram.

#### FeatureInput

A `FeatureInput` is simply a Python dictionary that contains information from both a Feature module and user-information processed by the `input_` module.  Here is a sample with the associated items tabulated; you may note fimilar `load_params` and `mat_props` from boilerpoint code:

```python

FeatureInput = {
    'Geometry': Geometry,                           # defined in Case      
    'Parameters': load_params,
    'Properties': mat_props,
    'Materials': materials,                         # set material order
    'Model': model,
    'Globals': None,                                # defined in models
}             
```

| Key | Value | Description | 
|:---|:-----|:-----------|
| `'Geometry'` | Geometry object | a `namedtuple` of geometry thicknesses |
|`'Parameters'`| load_params | loading parameters |
| `'Properties'` | mat_props | material properties, e.g. modulus, Poisson's ratio |
| `'Materials'` | materials index | ordered list of materials from a DataFrame index |
| `'Model'` | model str | a model name; string object |
| `'Globals'` | None | an optional placeholder for ubiquitous model variables |


#### LaminateModel

A `LaminateModel` is simply a `pandas` DataFrame that combines data processed by a `constructs` object and `theories` model.  Details of this object will be discussed further in the [constructs](components.ipynb#Core-Module:-constructs) section. Other "Laminate-like" objects include parent classes, `Laminate` and `Stack` objects.  Specifics for each object are detailed in corresponding docstrings.